# YOLOv4 training with Darknet using Colab GPU for Wrong Side Driving Detection

Done By,  
Sriram N C  
Srinandan K S  



Credit- The AI Guy, https://colab.research.google.com/drive/1_GdoqCJWXsChrOiY8sZMr_zbr_fH-0Fg?usp=sharing


In [ ]:
# clone darknet repo
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

In [ ]:
# verify CUDA
!/usr/local/cuda/bin/nvcc --version

In [ ]:
# make darknet (builds darknet so that you can then use the darknet executable file to run or train object detectors)
!make

In [ ]:
#pretrain weight cell above deleted since ours is a custom dataset
# define helper functions
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# use this to upload files
def upload():
  from google.colab import files
  uploaded = files.upload()
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# use this to download a file
def download(path):
  from google.colab import files
  files.download(path)

In [ ]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

In [ ]:
# cd back into the darknet folder to run detections
%cd darknet

In [ ]:
# this is where my datasets are stored within my Google Drive (I created a yolov4 folder to store all important files for custom training)
!ls /mydrive/yolo

In [ ]:
# copy over both datasets into the root directory of the Colab VM (comment out test.zip if you are not using a validation dataset)
!cp /mydrive/yolo/obj.zip ../
!cp /mydrive/yolo/test.zip ../

In [ ]:
# unzip the datasets and their contents so that they are now in /darknet/data/ folder
!unzip ../obj.zip -d data/
!unzip ../test.zip -d data/

In [ ]:
# upload the custom .cfg to cloud VM from Google Drive
!cp /mydrive/yolo/yolov4-obj.cfg ./cfg

In [ ]:
# upload the obj.names and obj.data files to cloud VM from Google Drive
!cp /mydrive/yolo/obj.names ./data
!cp /mydrive/yolo/obj.data  ./data

In [ ]:
# upload the generate_train.py and generate_test.py script to cloud VM from Google Drive
!cp /mydrive/yolo/generate_train.py ./
!cp /mydrive/yolo/generate_test.py ./

In [ ]:
!python generate_train.py
!python generate_test.py

In [ ]:
# verify that the newly generated train.txt and test.txt can be seen in our darknet/data folder
!ls data/

Download pre-trained weights for the convolutional layers.  

This step downloads the weights for the convolutional layers of the YOLOv4 network. By using these weights it helps your custom object detector to be way more accurate and not have to train as long. We don't have to use these weights but it will help our model converge and be accurate way faster.

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

In [ ]:
# train your custom detector! (uncomment %%capture below if you run into memory issues or your Colab is crashing)
# %%capture
!./darknet detector train data/obj.data cfg/yolov4-obj.cfg yolov4.conv.137 -dont_show -map

After training, we can observe a chart of how the model did throughout the training process by running the below command. It shows a chart of the average loss vs. iterations. For the model to be 'accurate', loss should be under 2.

In [ ]:
# show chart.png of how custom object detector did with training
imShow('chart.png')

In [ ]:
# OPTIONAL-need to set our custom cfg to train mode if accuracy is insufficient after testing
%cd cfg
!sed -i 's/batch=1/batch=64/' yolov4-obj.cfg
!sed -i 's/subdivisions=1/subdivisions=64/' yolov4-obj.cfg
%cd ..

/content/darknet/cfg
/content/darknet


In [ ]:
# OPTIONAL- resume training after crash or after testing(run cell above) gives inaccurate result
!./darknet detector train data/obj.data cfg/yolov4-obj.cfg /mydrive/yolo/backup/yolov4-obj_last.weights -dont_show

Once the above cell is run, the weights file is stored in the drive. Copy the links of these weights file to be used for prediction. The following cells are just used to test the weights generated.

In [ ]:
#OPTIONAL- To observe mean avg precision (MAP) if training done without '-map' flag
!./darknet detector map data/obj.data cfg/yolov4-obj.cfg /mydrive/yolo/backup/yolov4-obj_best.weights

In [ ]:
# need to set our custom cfg to test mode
%cd cfg
!sed -i 's/batch=64/batch=1/' yolov4-obj.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' yolov4-obj.cfg
%cd ..

/content/darknet/cfg
/content/darknet


In [ ]:
# run the custom detector with this command (upload an image to our google drive to test, thresh flag sets accuracy that detection must be in order to show it)
!./darknet detector test data/obj.data cfg/yolov4-obj.cfg /mydrive/yolo/backup/yolov4-obj_last.weights /mydrive/agera.jpeg -thresh 0.3
imShow('predictions.jpg')

In [ ]:
!./darknet detector demo data/obj.data cfg/yolov4-obj.cfg /mydrive/yolo/backup/yolov4-obj_last.weights -dont_show /mydrive/test85.mp4 -ext_output -i 0 -out_filename /mydrive/newresults3.avi